# Open Market 자동화
## Overview

### 인식: 인간처리에 좀 더 가깝게.

### 개발 리스트

  1. 복수 오픈마켓 주문정보 수집하여 메일로 알림
  2. 복수 오픈마켓 재고정보 수집하여 메일로 알림
  3. 복수 오픈마켓 부가세 신고자료 취합하여 엑셀로 정리
  
### 2개의 version

1. 브라우저 핸들링 (Selenium)
2. 공식 REST API

### Open Market List

- 지마켓 (http://www.gmarket.co.kr)
- 옥션 (http://www.auction.co.kr/
- 11번가 (https://www.11st.co.kr/html/main.html)
- 스마트스토어 (https://sell.smartstore.naver.com)



## Version1. 브라우저 핸들링 (selenium)

## 1-1. 복수 오픈마켓 주문정보 수집하여 메일로 알림

# <공통> (각 사이트마다)

1. 초기 설정 (브라우저 설정, 브라우저 위치 등)
2. 로그인 페이지 접속
3. 로그인 시도
4. 주문정보 확인 사이트로 이동
5. 주문정보 확인 필터링 걸기
6. 엑셀 다운로드 or crawling
7. 데이터 가공(엑셀 합치기 or HTML 만들기)
8. 메일 보내기

#### 11ST

##### (0) 로그인 페이지 접속

In [1]:
## 1. 초기 설정 (브라우저 설정, 브라우저 환경설정(header 등) )
"""
정말 많은 brower가 있습니다만,
본 예제에서는 OS에 독립적으로 실행되길 원하므로,
보편적인 Chrome을 사용하겠습니다.
만약 chrome driver가 없다면 아래 링크에서 설치하십시요.
- http://chromedriver.storage.googleapis.com/index.html
"""

from selenium import webdriver


# TODO: Option에 대한 설명 필요.
# options = None
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": '.'
})
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.implicitly_wait(30)


### 아래와 같은 많은 브라우저들이 있습니다.

# driver = webdriver.FireFox
# webdriver.FirefoxProfile

# driver = webdriver.Ie
# driver = webdriver.Opera

# driver = webdriver.PhantomJS

# driver = webdriver.Remote
# webdriver.DesiredCapabilities

# driver = webdriver.safairi

## ...



#### ESM Plus(Gmarket. & Auction)
##### (2) 로그인 페이지 접속

In [2]:
# 관리자 로그인 페이지 접속
login_url = "https://login.soffice.11st.co.kr/login/Login.page"

driver.get(login_url) # onload wait 포함.


##### (3) 로그인 시도

In [3]:
esm_userid = 'picturemall'
esm_password = 'hsh981129*'

# id_input 찾기.
id_input = driver.find_element_by_xpath('//*[@id="loginName"]')
# password_input 찾기.
password_input = driver.find_element_by_xpath('//*[@id="passWord"]')
# login_button 찾기.
login_button = driver.find_element_by_xpath('//*[@id="layBody"]/div/div/form/div[2]/div/div/fieldset/div/div/input')

actions = webdriver.ActionChains(driver)

actions.move_to_element(id_input).click().send_keys(esm_userid)
actions.move_to_element(password_input).click().send_keys(esm_password)
actions.move_to_element(login_button).click()
actions.perform()

actions.reset_actions()


##### (예외처리) [Password변경 요청.]

In [4]:
# 보류

#### 주문정보 확인 사이트로 이동

In [5]:
# is redirected to admin/home
# print(driver.current_url)
# print(driver.current_window_handle)
# print(driver.desired_capabilities)

# TODO: Driver.switch_to 객체에 대해 공부 필요..(문서에 없음.)

actions = webdriver.ActionChains(driver)

order_manage_button = driver.find_element_by_xpath('//*[@id="gnb2"]/a')
actions.move_to_element(order_manage_button).click()

new_order_button = driver.find_element_by_xpath('//*[@id="Gnb_ly2"]/li[1]/a')
actions.move_to_element(new_order_button).click()

actions.perform()
actions.reset_actions()




#### 주문정보 확인 필터링 걸기

In [7]:
### 우리는 최근 한달 신규 주문만 조회하겠습니다.

actions = webdriver.ActionChains(driver)

# 사실 피료 없다.
# order_tab = driver.find_element_by_xpath('//*[@id="tTDM105"]/a[2]')
# actions.move_to_element(order_tab).click()
# actions.perform()
# actions.reset_actions()

driver.switch_to.window(driver.window_handles[0])
driver.switch_to.default_content()
print(driver.find_element_by_xpath('//*[@id="Content_ifrm_6201"]'))
driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="Content_ifrm_6201"]'))

# print(driver.page_source)

deliver_reserved = driver.find_element_by_xpath('//*[@id="ext-gen6"]/div[4]/div[1]/div[3]/ul/li[2]/a')
actions.move_to_element(deliver_reserved).click()


actions.perform()
actions.reset_actions()

<selenium.webdriver.remote.webelement.WebElement (session="956e5e907029967d4a839d8867e886a4", element="0.27454415419087863-3")>


#### 엑셀 다운로드 or crawling

In [8]:
download_excel = driver.find_element_by_xpath('//*[@id="ext-gen6"]/div[4]/div[1]/div[4]/div/a')
actions.move_to_element(download_excel).click()

actions.perform()
actions.reset_actions()

# https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.alert



#### 데이터 가공(엑셀 합치기 or HTML 만들기)
#### 메일 보내기